# Bitcoin News Sentiment Analysis (LiteLLM, Modular, Table Display)

This notebook fetches the latest Bitcoin news, analyzes sentiment using an LLM (via LiteLLM), and displays the results in a table.

**Instructions:**
- Make sure you have a `.env` file with your API keys (NEWS_API_KEY, OPENAI_API_KEY, etc).
- Run the setup cell below to install dependencies.
- You can change the LLM provider in the configuration cell.

In [1]:
%pip install -q litellm requests python-dotenv pandas tiktoken ipywidgets plotly

Note: you may need to restart the kernel to use updated packages.


## Imports and Configuration

In [2]:
import os
import json
import hashlib
from datetime import datetime, timezone
from pathlib import Path
from typing import List, Dict, Optional
from dataclasses import dataclass

import requests
import pandas as pd
from dotenv import load_dotenv
import tiktoken
from litellm import completion

# For table display
from IPython.display import display, HTML
import ipywidgets as widgets

# For optional visualization
import plotly.express as px

### Load environment variables and set up config

In [3]:
load_dotenv()

NEWS_API_KEY = os.getenv('NEWS_API_KEY')
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
TOGETHER_API_KEY = os.getenv('TOGETHER_API_KEY')
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')
HUGGINGFACE_API_KEY = os.getenv('HUGGINGFACE_API_KEY')

PROVIDER = 'together_ai'  # Change to 'openai', 'huggingface', etc.
MODEL_ID = 'mistralai/Mixtral-8x7B-Instruct-v0.1'

PROVIDER_CONFIGS = {
    'together_ai': {
        'model': 'together_ai/mistralai/Mixtral-8x7B-Instruct-v0.1',
        'api_key': TOGETHER_API_KEY,
    },
    'openai': {
        'model': 'gpt-3.5-turbo',
        'api_key': OPENAI_API_KEY,
    },
    'huggingface': {
        'model': 'huggingface/HuggingFaceH4/zephyr-7b-beta',
        'api_key': HUGGINGFACE_API_KEY,
    },
    'gemini': {
        'model': 'gemini-pro',
        'api_key': GOOGLE_API_KEY,
    }
}

PROMPT_TEMPLATE = (
    "You are a financial sentiment analyst.\n"
    "Classify the sentiment of the following Bitcoin news article as one of 'Positive', 'Neutral', or 'Negative'. Respond with ONLY the full word (not abbreviated).\n\n"
    "Article: {text}\n\nSentiment:"
)

## Helper Functions

In [4]:
def fetch_bitcoin_news(page_size: int = 10) -> List[Dict]:
    url = 'https://newsapi.org/v2/everything'
    params = {
        'q': 'Bitcoin',
        'language': 'en',
        'sortBy': 'publishedAt',
        'pageSize': page_size,
        'apiKey': NEWS_API_KEY,
    }
    r = requests.get(url, params=params, timeout=15)
    r.raise_for_status()
    return r.json().get('articles', [])

def count_tokens(text: str) -> int:
    encoding = tiktoken.get_encoding('cl100k_base')
    return len(encoding.encode(text))

def truncate_text(text: str, max_tokens: int = 1000) -> str:
    if count_tokens(text) <= max_tokens:
        return text
    sentences = text.split('. ')
    truncated = []
    current_tokens = 0
    for sentence in sentences:
        sentence_tokens = count_tokens(sentence)
        if current_tokens + sentence_tokens <= max_tokens:
            truncated.append(sentence)
            current_tokens += sentence_tokens
        else:
            break
    return '. '.join(truncated) + '.'

def classify_sentiment(text: str, provider: str = PROVIDER) -> str:
    truncated_text = truncate_text(text)
    prompt = PROMPT_TEMPLATE.format(text=truncated_text.strip())
    config = PROVIDER_CONFIGS[provider]
    response = completion(
        model=config['model'],
        messages=[{'role': 'user', 'content': prompt}],
        temperature=0,
        max_tokens=1,
        api_key=config.get('api_key'),
    )
    result = response.choices[0].message.content.strip().split()[0]
    return result


## Run Sentiment Analysis on Latest News

In [5]:
articles = fetch_bitcoin_news(page_size=10)
rows = []
for art in articles:
    headline = art.get('title', '').strip()
    content = art.get('description', '') or art.get('content', '')
    if not content:
        continue
    sentiment = classify_sentiment(content)
    rows.append({
        'publishedAt': art.get('publishedAt'),
        'headline': headline,
        'sentiment': sentiment,
        'url': art.get('url'),
    })
df = pd.DataFrame(rows)
df['publishedAt'] = pd.to_datetime(df['publishedAt'])
df = df.sort_values('publishedAt', ascending=False)
df.reset_index(drop=True, inplace=True)
df.head()

,publishedAt,headline,sentiment,url
0,2025-05-16 08:10:15+00:00,Refer-a-Friend and You BOTH get a $20 BTC Bonu...,Pos,https://www.ozbargain.com.au/node/906089
1,2025-05-16 08:07:00+00:00,Cetera Investment Advisers Decreases Holdings ...,Ne,https://www.etfdailynews.com/2025/05/16/cetera...
2,2025-05-16 08:00:53+00:00,"Mercer Global Advisors Inc. ADV Takes $312,000...",Ne,https://www.etfdailynews.com/2025/05/16/mercer...
3,2025-05-16 08:00:49+00:00,Bitcoin Could See Short-Term Holder Selloff Ar...,Ne,http://www.newsbtc.com/bitcoin-news/bitcoin-sh...
4,2025-05-16 08:00:28+00:00,Monero: Will an 82% rally in 40 days help XMR ...,Ne,https://ambcrypto.com/monero-will-an-82-rally-...


## Display Results as Interactive Table

In [6]:
def make_clickable(val):
    return f'<a href="{val}" target="_blank">Link</a>' if pd.notnull(val) else ''

display(HTML(df.to_html(escape=False, formatters={'url': make_clickable}, index=False)))

publishedAt,headline,sentiment,url
2025-05-16 08:10:15+00:00,Refer-a-Friend and You BOTH get a $20 BTC Bonus (Plus BOTH get $5 in Reward Points) @ Coinjar,Pos,Link
2025-05-16 08:07:00+00:00,"Cetera Investment Advisers Decreases Holdings in MARA Holdings, Inc. (NASDAQ:MARA)",Ne,Link
2025-05-16 08:00:53+00:00,"Mercer Global Advisors Inc. ADV Takes $312,000 Position in Bitwise Bitcoin ETF (NYSEARCA:BITB)",Ne,Link
2025-05-16 08:00:49+00:00,"Bitcoin Could See Short-Term Holder Selloff Around These Levels, Analyst Says",Ne,Link
2025-05-16 08:00:28+00:00,Monero: Will an 82% rally in 40 days help XMR reach $489?,Ne,Link
2025-05-16 07:56:47+00:00,Bitcoin hitting $220K 'reasonable' in 2025 says gold-based forecast,Pos,Link
2025-05-16 07:49:05+00:00,"Mercer Global Advisors Inc. ADV Makes New $293,000 Investment in CompoSecure, Inc. (NASDAQ:CMPO)",Ne,Link
2025-05-16 07:48:38+00:00,Bitcoin Bulls Face $120M Challenge in Extending 'Stair-Step' Uptrend,Pos,Link
2025-05-16 07:43:20+00:00,Comerica Bank Sells 439 Shares of Grayscale Bitcoin Trust (NYSEARCA:GBTC),Ne,Link
2025-05-16 07:41:26+00:00,Ukraine Strategic Bitcoin Reserve Bill Reportedly In Final Stages,Pos,Link


## Optional: Visualize Sentiment Distribution

In [7]:
fig = px.histogram(df, x='sentiment', color='sentiment', title='Sentiment Distribution')
fig.show()